### The Problem
**Locality of reference principle** states recently requested data is likely to be requested again. If that data is stored in slow to access disc storage, the retrieval will be slow.

### Cache
A cache is like short-term memory: it has a limited amount of space, but is typically faster than the original data source and contains the most recently accessed items.  
<img src="images/caching.png">